# Group Methods

_This is a Jupyter Notebook file, make sure to run all the cells in the presented order to avoid any errors_ 

This tutorial details how to work with groups in ETABS using `PyCSI`. This example uses an ETABS model; however, connecting to a SAP2000 or SAFE model is done by changing only the ETABSmodel class for the desired software.

## Opening example model

See the [Basic Functions](2_Basic_functions.ipynb) tutorial for details of the following code

In [ ]:
# Python packages imports
import os
import sys

###################################################################
# The following lines are only necessary for these tutorials
from pathlib import Path
pyCSI_dir = Path.cwd().parents[2]
os.chdir(pyCSI_dir)
###################################################################

# PyCSI import
import pyCSI

# example_model location 
# DO NOT MODIFY THE FOLLOWING LINES
VERSION = 21
EXAMPLE_LOCATION = pyCSI_dir.joinpath('docs', 'tutorials', 'resources', 'example_model')
EXAMPLE_NAME = 'pyCSI_example - v' + str(VERSION) + '.EDB'
EXAMPLE_FILE = EXAMPLE_LOCATION.joinpath(EXAMPLE_NAME)
open_file = False  # Flag to be used in the next step

model = pyCSI.ETABSModel(version=VERSION)

try:
    # Calling get_model() with no arguments connects to an active model
    model.get_model()
    file_name = model.get_file_name()  # Get the name of the connected model

    # Check that the connected model is the pyCSI_example - v21.EDB file
    # If not, set the flag to open the model in the next step
    if not file_name == EXAMPLE_NAME:
        print('Not connected to Example Model')
        open_file = True
    
except pyCSI.APIConnectionError:
    # If no active model is found, an Attribute error is raised
    # Set the flag to open the model in the next step
    print('No active model found')
    open_file = True

if open_file:
    print('Opening example model')
    model.get_model(active_model=False, file_location=EXAMPLE_FILE)

## Group Property

`PyCSI` provides the `.groups` property. This property gives access to different methods to work with groups within ETABS. The following sections detail the available methods.


### Create Group

Use the `.groups.create(group_name)` method to define a new, empty group in the model.

In [ ]:
GROUP = 'PyCSI Group'
model.groups.create(group_name=GROUP)
print(f'{GROUP = } created.')


### Get all defined groups in the model

Use the `.groups.get_names()` method to get a list with all the group names currently defined in the model.

In [ ]:
group_names = model.groups.get_names()
print('Group Names:\n', *group_names, sep= '\n')

### Adding objects to a group

`PyCSI` provides different methods to add or remove objects to a group. To achieve this, you must first identify the objects you want to assign to the group.

#### What is an object?

CSI software uses objects to represent different structural elements in an analysis model.

The main object types are:
* `Point` Objects. This represents the nodes in the model.
* `Frame` Objects. This represents the beams, columns and braces in the model.
* `Area` Objects. This represents the floors and walls in the model.

##### How to identify objects in the model?

Objects in a model are identified in two different ways:
1. Object `Unique Name`: Every object has a name within the model, unique for each object type (point, frame, area or link).
   
2. Object `Label`: Every object is identified by its location in plan, and this is called the label. Objects in different stories that have the same location in plan usually have the same label. To identify an object by its label, it is necessary to specify its label and story.

#### Adding objects to a group by its Unique Name

To add an object to a group by its unique name, it is  necessary to identify the object by its unique name and object type.

<center><img src="..\..\images\tutorials\Object_Unique_name.png" alt="Object Unique Name" width="600"/></center>

##### Add a single object to a group

Use the `.groups.add_object_from_name(unique_name, object_type, group_name[, replace_group, remove])` method to add a single object to the specified group.

Arguments:
* unique_name -- Object unique name in string format
* object_type -- Object type in string format. Valid values are:
  * frame
  * area
  * joint
  * link
* group_name -- Name of the group to be modified

Keyword Arguments:
* replace_group -- Optional. If True, the specified objects will replace the group. Otherwise, objects will be added to the group (default: {False})
* remove -- Optional. If True, objects will be removed from specified group (default: {False})
        

In [ ]:
# Identify the object by its unique name and object type
NAME = '222'
TYPE = 'frame'
GROUP = 'PyCSI Group'

model.groups.add_object_from_name(NAME, TYPE, GROUP)
print(f'{TYPE} object {NAME} added to group: {GROUP}')

##### Add multiple objects to a group

To add multiple objects to a group in a single call the objects must be passed to `PyCSI` with one of the following methods:

##### Objects in a Pandas DataFrame

Use the `.groups.add_objects_from_dataframe_names(objects, group_name[, replace_group, remove])` method to add multiple objects to a group.

The `objects` argument must be a two-column DataFrame, where each row must contain the object's unique name and object type.

See [above](#add-a-single-object-to-a-group) for the full argument list

In [ ]:
# Adding multiple objects from a pandas DataFrame

# See Tables tutorial for details of the next function
frame_assignments = model.tables.get_table_dataframe('Frame Assignments - Summary')

# Take the Unique Name column from Frame Assignments table and add a second column with the object type
frame_unique_names = frame_assignments['UniqueName'].to_frame()
frame_unique_names['Type'] = 'frame'
print('Adding the following elements to group')
display(frame_unique_names)

# Add objects to Group
GROUP = 'Frames from Names'
model.groups.add_objects_from_dataframe_names(frame_unique_names, GROUP, replace_group=True)
print(f'Objects from DataFrame added to group: {GROUP}')

Additionally, if all the objects you are adding to the group are of the same type, you can use the `.groups.add_objects_of_same_type_from_dataframe_names(objects, group_name, object_type[, replace_group, remove])`.

Here, the `objects` argument is a single-column DataFrame containing only the object's unique name and the `object_type` argument must be one of the valid object types.

In [ ]:
# Adding multiple objects of the same type from a pandas DataFrame

# See Tables tutorial for details of the next function
area_assignments = model.tables.get_table_dataframe('Area Assignments - Summary')

# Take the Unique Name column from Area Assignments table
area_assignments = area_assignments['UniqueName'].to_frame()
print('Adding the following elements to group')
display(area_assignments)

# Add objects to Group
GROUP = 'Areas from Names'
TYPE = 'area'
model.groups.add_objects_of_same_type_from_dataframe_names(area_assignments, GROUP, object_type=TYPE)
print(f'{TYPE} objects from DataFrame added to group: {GROUP}')

   
##### Objects in the Clipboard

_Before using these methods you need to copy into your clipboard the list of objects to add to the group. You can grab this list from a text or excel file. See the [resources](resources) folder for example of file contents._

Use the `.groups.add_objects_from_clipboard_names(group_name[, replace_group, remove])` method to add multiple objects to a group. Open the [objects_unique_names.txt](resources\objects_unique_names.txt) or [objects_unique_names.xlsx](resources\objects_unique_names.xlsx) file and copy its content before running the following cell.

In [ ]:
# Add objects to Group
GROUP = 'Objects from Clipboard Names'
model.groups.add_objects_from_clipboard_names(GROUP)
print(f'Objects from Clipboard added to group: {GROUP}')

Additionally, if all the objects you are adding to the group are of the same type, you can use the `.groups.add_objects_of_same_type_from_clipboard_names(group_name, object_type[, replace_group, remove])`.Open the [objects_of_same_type_unique_names.txt](resources\objects_of_same_type_unique_names.txt) or [objects_unique_names.xlsx](resources\objects_unique_names.xlsx) file and copy its content before running the following cell.

In [ ]:
# Add objects to Group
GROUP = 'Points from Clipboard Names'
TYPE = 'joint'
model.groups.add_objects_of_same_type_from_clipboard_names(GROUP, object_type=TYPE)
print(f'{TYPE} objects from Clipboard added to group: {GROUP}')

#### Adding objects to a group by its Label

To add an object to a group by its label, it is  necessary to identify the object by its label and story.

<center><img src="..\..\images\tutorials\Object_Label.png" alt="Object Label" width="600"/></center>

##### Add a single object to a group

Use the `.groups.add_object_from_label(label, story, group_name[, replace_group, remove])` method to add a single object to the specified group.

Arguments:
* label -- Object label in string format
* story -- Object story in string format
* group_name -- Name of the group to be modified

Keyword Arguments:
* replace_group -- Optional. If True, the specified objects will replace the group. Otherwise, objects will be added to the group (default: {False})
* remove -- Optional. If True, objects will be removed from specified group (default: {False})
        

In [ ]:
# Identify the object by its unique name and object type
LABEL = 'B49'
STORY = 'Roof'
GROUP = 'PyCSI Group'

model.groups.add_object_from_label(LABEL, STORY, GROUP)
print(f'{LABEL} object {NAME} added to group: {GROUP}')

##### Add multiple objects to a group

To add multiple objects to a group in a single call the objects must be passed to `PyCSI` with one of the following methods:

##### Objects in a Pandas DataFrame

Use the `.groups.add_objects_from_dataframe_labels(objects, group_name[, replace_group, remove])` method to add multiple objects to a group.

The `objects` argument must be a two-column DataFrame, where each row must contain the object's label and story.

See [above](#adding-objects-to-a-group-by-its-label) for the full argument list

In [ ]:
# Adding multiple objects from a pandas DataFrame

# See Tables tutorial for details of the next function
frame_assignments = model.tables.get_table_dataframe('Frame Assignments - Summary')


# Take the Label and Story columns
frame_labels = frame_assignments[['Label', 'Story']]
display(frame_labels)
                                 
# Add objects to Group
GROUP = 'Frames from Labels'
model.groups.add_objects_from_dataframe_labels(frame_labels, GROUP, replace_group=True)
print(f'Objects from DataFrame added to group: {GROUP}')


   
##### Objects in the Clipboard

_Before using these methods you need to copy into your clipboard the list of objects to add to the group. You can grab this list from a text or excel file. See the [resources](resources) folder for example of file contents._

Use the `.groups.add_objects_from_clipboard_labels(group_name[, replace_group, remove])` method to add multiple objects to a group. Open the [objects_labels.txt](resources\objects_labels.txt) or [objects_labels.xlsx](resources\objects_labels.xlsx) file and copy its content before running the following cell.

In [ ]:
# Add objects to Group
GROUP = 'Objects from Clipboard Labels'
model.groups.add_objects_from_clipboard_labels(GROUP)
print(f'Objects from Clipboard added to group: {GROUP}')

## Next -> [Example Use](5_Example_use.ipynb)

**REMEMBER to shut down the Kernel before leaving**

Click `Kernel -> Shut Down Kernel`

### Contact

For questions or comments please reach out to:

* Luis Pancardo: [lpancardo@degenkolb.com](lpancardo@degenkolb.com)<br/>
* Daniel Gaspar:  [dgaspar@degenkolb.com](dgaspar@degenkolb.com)<br/>
